In [ ]:
#fixed seed to get similar score
from transformers import set_seed
set_seed(42)

In [ ]:
import os
import gc
import time
import warnings

import pandas as pd
import polars as pl

import torch
import kaggle_evaluation.aimo_2_inference_server

pd.set_option('display.max_colwidth', None)
cutoff_time = time.time() + (4 * 60 + 45) * 60

# Load Model

In [ ]:
from vllm import LLM, SamplingParams

warnings.simplefilter('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

def clean_memory(deep=False):
    gc.collect()
    if deep:
        ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

llm_model_pth = '/kaggle/input/m/shelterw/qwen2.5/transformers/qwq-32b-preview-awq/1'

llm = LLM(
    llm_model_pth,
    #dtype="half",                -> Changed this
    #max_num_seqs=128,            -> Changed this
    max_model_len=32768,#4096*10,         
    trust_remote_code=True,     
    tensor_parallel_size=4,      
    gpu_memory_utilization=0.96, 
)

In [ ]:
tokenizer = llm.get_tokenizer()

# Utlities

In [ ]:
import re

#prompts
thoughts = [
    """Act as five distinct mathematical experts specializing in tricky math problems, algebra, combinatorics, geometry, and number theory. Each expert will:
    Identify their specialization and write down their initial step, explaining their reasoning thoroughly.
    Share their response with the group for review.
    Evaluate the responses of their peers, assigning a score from 1 to 5:
    1: Highly unlikely.
    5: Highly likely.
    Experts who receive consistent low scores indicating errors will exit the discussion.    
    Proceed step-by-step, refining the solution collaboratively, until the problem is fully addressed.
    At the conclusion of the process, consolidate the insights from all experts to provide either a consensus solution or your best-informed answer, formatted as \\boxed{}.""",
    """Use a chain of reasoning to solve the given math problem. Then, simulate a debate between two groups of five experts each:
    Group A:
    Composed of experts specializing in tricky math problems, algebra, combinatorics, and number theory.
    Argues in support of the proposed solution, explaining why it is correct and addressing potential doubts.
    Group B:
    Composed of similarly specialized experts.
    Critiques the proposed solution, aiming to disprove it and present an alternate solution.
    The debate will proceed until one group successfully convinces the other of its stance. At the conclusion of the process present a consensus solution, and put the answer in \\boxed{}.""",
    'Solve the following problem using concise and clear reasoning by placing the answer in \\boxed{}.',
    'You are a helpful and reflective maths assistant, please reason step by step to put the answer in \\boxed{}.',
    """As an expert in tricky math problems, algebra, combinatorics, and number theory, solve the given question using a chain of thought approach. Follow these steps:
    Reason through the problem step-by-step, ensuring logical consistency at each stage.
    Challenge and doubt your solution at every step, identifying potential errors or alternative interpretations.
    Solve the problem using five distinct methods, leveraging your expertise in:
    Algebraic manipulation.
    Combinatorial analysis.
    Number theory principles.
    Geometric or graphical visualization.
    Generalization and special-case testing.
    Compare the outcomes of all five methods, evaluate their validity, and select the most likely correct solution.
    Present the final answer in the format: \boxed{solution}
    
    """]

#create single prompt
def make_next_prompt(text,round_idx):
    default_prompt = thoughts[(round_idx+1)%len(thoughts)]
    default_python_code = f"print('{default_prompt}')"
    return default_python_code

#extract python code from response
def extract_python_code(text):
    pattern = r'```python\s*(.*?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    if matches:
        ans = "\n\n".join(matches)
        #print(f'Extracted python code: {ans}')
        return ans
    return ""

#extract all code segments
def extract_python_code_list(text):
    pattern = r'```python\s*(.*?)\s*```'
    ans=[]
    matches = re.findall(pattern, text, re.DOTALL)
    for m in matches:
        ans.append(m)
    return ans

#process the code
def process_python_code(query):
    query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
    current_rows = query.strip().split("\n")
    new_rows = []
    for row in current_rows:
        new_rows.append(row)
    ans = "\n".join(new_rows)
    print(f'Processed python code: {ans}')
    return ans

import re

#extract the answer from the boxes
def extract_boxed_texts(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return []
    ans = []
    for content in matches:
        if content.isdigit():
            num = int(content)
        else:
            nums = re.findall(r'\d+', content)
            if not nums:
                continue
            num = int(nums[-1])
        ans.append(num % 1000)
    return ans
    
#extract the integer answer modulo 1000 from the boxes
def extract_boxed_text(text):
    pattern = r'oxed{(.*?)}'
    matches = re.findall(pattern, text)
    if not matches:
        return -1
    content = matches[0]
    if content.isdigit():
        num = int(content)
    else:
        nums = re.findall(r'\d+', content)
        if not nums:
            return -1
        num = int(nums[-1])
    return num % 1000

#select the final answer based on the frequency (majoity voting)
from collections import Counter
def select_answer(answers):
    valid_answers = []
    for answer in answers:
        try:
            if int(answer) == float(answer):
                if 1 < int(answer) < 999 and int(answer) % 100 > 0:
                    valid_answers.append(int(answer))
        except:
            pass
    if not valid_answers:
        return 49
    _, answer = sorted([(v,k) for k,v in Counter(valid_answers).items()], reverse=True)[0]
    return answer%1000

In [ ]:
import os
import tempfile
import subprocess

#Python REPL to execute code. taken and modified from NuminaMath Solution
#NuminaMath solution can be found here : https://www.kaggle.com/code/lewtun/numina-1st-place-solution

class PythonREPL:
    def __init__(self, timeout=8):
        self.timeout = timeout

    def __call__(self, query):
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            
            try:
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
            except subprocess.TimeoutExpired:
                return False, f"Execution timed out after {self.timeout} seconds."

            stdout = result.stdout.strip()
            stderr = result.stderr.strip()

            if result.returncode == 0:
                return True, stdout
            else:
                # Process the error message to remove the temporary file path
                # This makes the error message cleaner and more user-friendly
                error_lines = stderr.split("\n")
                cleaned_errors = []
                for line in error_lines:
                    if temp_file_path in line:
                        # Remove the path from the error line
                        line = line.replace(temp_file_path, "<temporary_file>")
                    cleaned_errors.append(line)
                cleaned_error_msg = "\n".join(cleaned_errors)
                # Include stdout in the error case
                combined_output = f"{stdout}\n{cleaned_error_msg}" if stdout else cleaned_error_msg
                return False, combined_output

In [ ]:
# ... (Code Set A remains the same until this point)

# Python REPL and code execution utilities
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()


def execute_completion(executor, completion, return_status, last_code_block):
    executions = re.findall(r"```python(.*?)```", completion, re.DOTALL)
    if len(executions) == 0:
        return completion, False if return_status else completion
    if last_code_block:
        executions = [executions[-1]]
    outputs = []
    successes = []
    for code in executions:
        success = False
        for lib in ("subprocess", "venv"):
            if lib in code:
                output = f"{lib} is not allowed"
                outputs.append(output)
                successes.append(success)
                continue
        try:
            success, output = executor(code)
        except TimeoutError as e:
            print("Code timed out")
            output = e
        if not success and not return_status:
            output = ""
        outputs.append(output)
        successes.append(success)
    output = str(outputs[-1]).strip()
    success = successes[-1]
    if return_status:
        return output, success
    return output


def postprocess_completion(text, return_status, last_code_block):
    executor = PythonREPL()
    result = execute_completion(executor, text, return_status=return_status, last_code_block=last_code_block)
    del executor
    return result


# Rest of Code Set A remains the same

In [ ]:
#sanity check
list_of_texts = [
    tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True
    )
    for messages in [[{"role": "user", "content": "hi"}]]
]

# Text Generation Functions

In [ ]:
"""# ... (rest of the code remains the same)

# SC-TIR control flow (from code set 1)
def process_code(sample, restart_on_fail, last_step, check_last_n_chars=100):
    gen_text = sample["gen_texts"]
    num_python_blocks = len(re.findall(r"```python(.*?)```", gen_text, re.DOTALL))
    region_to_check = gen_text[-check_last_n_chars:]
    if num_python_blocks == 0:
        if restart_on_fail:
            print("no code has ever been generated, RESTARTING")
            sample["gen_texts"] = sample["text"]
        else:
            print("no code has ever been generated, STOP")
            sample["should_prune"] = True
            sample["has_code"] = False
        return sample
    if not gen_text.endswith("```output\n") and ("answer is" in region_to_check or "\\boxed" in region_to_check):
        num_output_blocks = len(re.findall(r"```output(.*?)```", gen_text, re.DOTALL))
        if num_output_blocks == 0:
            print("The model hallucinated the code answer")
            sample["should_prune"] = True
            return sample
        if "boxed" in region_to_check:
            try:
                answer = normalize_answer(extract_boxed_answer(region_to_check))
            except Exception:
                answer = "-1"
        else:
            answer = normalize_answer(region_to_check)
        sample["model_answers"] = answer
        return sample
    if last_step:
        return sample
    if not gen_text.endswith("```output\n"):
        print("warning: output block not found: ", gen_text[-40:])
        if restart_on_fail:
            sample["gen_texts"] = sample["text"]
        else:
            sample["should_prune"] = True
        return sample
    code_result, _ = postprocess_completion(gen_text, return_status=True, last_code_block=True)
    truncation_limit = 200
    if len(code_result) > truncation_limit:
        code_result = code_result[:truncation_limit] + " ... (output truncated)"
    sample["gen_texts"] = gen_text + f"{code_result}\n```"
    return sample

# ... (rest of the code remains the same)

# Text Generation Functions
#define the sampling parameters
sampling_params = SamplingParams(
    temperature=1.0,              # Controls randomness in generation: higher values (e.g., 1.0) produce more diverse output.
    min_p=0.01,                   # Minimum cumulative probability for nucleus sampling, filtering out unlikely tokens.
    skip_special_tokens=True,     
    # max_tokens=1800,            
    max_tokens=32768,             # Sets a very high limit for token generation to handle longer outputs.
    # stop=["```output"],       
)

#generate prompts in batch
def batch_message_generate(list_of_messages) -> list[list[dict]]:
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]
    
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    
    for messages, single_request_output in zip(list_of_messages, request_output):
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})
        print(messages[-1])

    # Apply SC-TIR control flow
    for i, messages in enumerate(list_of_messages):
        sample = {
            "gen_texts": messages[-1]['content'],
            "text": list_of_texts[i],
            "prompt": question
        }
        sample = process_code(sample, restart_on_fail=True, last_step=False)
        list_of_messages[i][-1]['content'] = sample["gen_texts"]

    return list_of_messages

# ... (rest of the code remains the same)"""

In [ ]:

# Post-processing and solution extraction utilities
import re

def extract_boxed_answer(text):
    def last_boxed_only_string(text):
        idx = text.rfind("\\boxed")
        if idx < 0:
            idx = text.rfind("\\fbox")
            if idx < 0:
                return None
        i = idx
        right_brace_idx = None
        num_left_braces_open = 0
        while i < len(text):
            if text[i] == "{":
                num_left_braces_open += 1
            if text[i] == "}":
                num_left_braces_open -= 1
                if num_left_braces_open == 0:
                    right_brace_idx = i
                    break
            i += 1
        if right_brace_idx is None:
            return None
        return text[idx : right_brace_idx + 1]

    def remove_boxed(boxed):
        left = "\\boxed{"
        try:
            assert boxed[: len(left)] == left
            assert boxed[-1] == "}"
            length = len(left)
            return boxed[length:-1]
        except Exception:
            return None

    boxed = last_boxed_only_string(text)
    if boxed is None:
        return None
    answer = remove_boxed(boxed)
    return answer


def normalize_answer(answer):
    match = re.search(r"(.*?)Problem:", answer, flags=re.S)
    if match:
        answer = match.group(1)
    subs = [("an ", ""), ("a ", ""), (".$", "$"), ("\\$", ""), (r"\ ", ""), (" ", ""), ("mbox", "text"), (",\\text{and}", ","), ("\\text{and}", ","), ("\\text{m}", "\\text{}"), ("\\le", "<")]
    remove = ["square", "ways", "integers", "dollars", "mph", "inches", "ft", "hours", "km", "units", "\\ldots", "sue", "points", "feet", "minutes", "digits", "cents", "degrees", "cm", "gm", "pounds", "meters", "meals", "edges", "students", "childrentickets", "multiples", "\\text{s}", "\\text{.}", "\\text{\ns}", "\\text{}^2", "\\text{}^3", "\\text{\n}", "\\text{}", r"\mathrm{th}", r"^\circ", r"^{\circ}", r"\;", r",\!", "{,}", '"', "\\dots", "\n", "\r", "\f", "\%"]
    sub_patterns = [r"(\\text\{)(.*?)(\})", r"(\\textbf\{)(.*?)(\})", r"(\\overline\{)(.*?)(\})", r"(\\boxed\{)(.*)(\})"]
    split_patterns = [r"finalansweris(.*)", r"answer?is:?(.*)", r"oxed\{(.*?)\}", r"\$(.*?)\$"]
    for before, after in subs:
        answer = answer.replace(before, after)
    for expr in remove:
        answer = answer.replace(expr, "")
    for pattern in sub_patterns:
        answer = re.sub(pattern, "\\2", answer)
    for pattern in split_patterns:
        if len(re.findall(pattern, answer)) > 0:
            answer = re.findall(pattern, answer)[-1]
    answer = answer.strip()
    if "rac" in answer and "\\frac" not in answer:
        answer = answer.replace("rac", "\\frac")
    answer = re.sub(r"(frac)([^{])(.)", "frac{\\2}{\\3}", answer)
    answer = re.sub(r"(sqrt)([^{])", "sqrt{\\2}", answer)
    answer = answer.replace("$", "")
    if answer.replace(",", "").isdigit():
        answer = answer.replace(",", "")
    return answer

# Rest of Code Set A remains the same

In [ ]:
"""def process_code(sample, restart_on_fail, last_step, check_last_n_chars=100,
                   normalize_answer=None, extract_boxed_answer=None, postprocess_completion=None):
    gen_text = sample["gen_texts"]
    num_python_blocks = len(re.findall(r"```python(.*?)```", gen_text, re.DOTALL))
    region_to_check = gen_text[-check_last_n_chars:]
    if num_python_blocks == 0:
        if restart_on_fail:
            print("no code has ever been generated, RESTARTING")
            sample["gen_texts"] = sample["prompt"]
        else:
            print("no code has ever been generated, STOP")
            sample["should_prune"] = True
            sample["has_code"] = False
        return sample
    if not gen_text.endswith("```output\n") and ("answer is" in region_to_check or "\\boxed" in region_to_check):
        num_output_blocks = len(re.findall(r"```output(.*?)```", gen_text, re.DOTALL))
        if num_output_blocks == 0:
            print("The model hallucinated the code answer")
            sample["should_prune"] = True
            return sample
        if "boxed" in region_to_check:
            try:
                answer = normalize_answer(extract_boxed_answer(region_to_check))
            except Exception:
                answer = "-1"
        else:
            answer = normalize_answer(region_to_check)
        sample["model_answers"] = answer
        return sample
    if last_step:
        return sample
    if not gen_text.endswith("```output\n"):
        print("warning: output block not found: ", gen_text[-40:])
        if restart_on_fail:
            sample["gen_texts"] = sample["prompt"]
        else:
            sample["should_prune"] = True
        return sample
    code_result, _ = postprocess_completion(gen_text, return_status=True, last_code_block=True)
    truncation_limit = 200
    if len(code_result) > truncation_limit:
        code_result = code_result[:truncation_limit] + " ... (output truncated)"
    sample["gen_texts"] = gen_text + f"{code_result}\n```"
    return sample


def batch_message_generate(list_of_messages) -> list[list[dict]]:
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]
    
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    
    for messages, single_request_output in zip(list_of_messages, request_output):
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})
        print(messages[-1])

    # Apply SC-TIR control flow
    for i, messages in enumerate(list_of_messages):
        sample = {
            "gen_texts": messages[-1]['content'],
            "prompt": messages[0]['content'],  # Use the original prompt
            "text": list_of_texts[i],
            "should_prune": False,
            "has_code": True,
            "model_answers": []
        }
        # Define missing functions or import them
        def normalize_answer(answer):
            return answer
        
        def extract_boxed_answer(text):
            pattern = r'oxed{(.*?)}'
            matches = re.findall(pattern, text)
            if matches:
                return matches[0]
            return ""
        
        def postprocess_completion(gen_text, return_status=False, last_code_block=False):
            return gen_text, ""

        sample = process_code(sample, restart_on_fail=True, last_step=False, 
                                 normalize_answer=normalize_answer, 
                                 extract_boxed_answer=extract_boxed_answer, 
                                 postprocess_completion=postprocess_completion)
        list_of_messages[i][-1]['content'] = sample["gen_texts"]

    return list_of_messages"""

In [ ]:
"""#define the sampling parameters
sampling_params = SamplingParams(
    temperature=1.0,              # Controls randomness in generation: higher values (e.g., 1.0) produce more diverse output.
    min_p=0.01,                   # Minimum cumulative probability for nucleus sampling, filtering out unlikely tokens.
    skip_special_tokens=True,     
    # max_tokens=1800,            
    max_tokens=32768,             # Sets a very high limit for token generation to handle longer outputs.
    # stop=["```output"],       
)

#generate prompts in batch
def batch_message_generate(list_of_messages) -> list[list[dict]]:
    list_of_texts = [
        tokenizer.apply_chat_template(
            conversation=messages,
            tokenize=False,
            add_generation_prompt=True
        )
        for messages in list_of_messages
    ]
    
    request_output = llm.generate(
        prompts=list_of_texts,
        sampling_params=sampling_params,
    )
    
    for messages, single_request_output in zip(list_of_messages, request_output):
        messages.append({'role': 'assistant', 'content': single_request_output.outputs[0].text})
        print(messages[-1])

    return list_of_messages"""

In [ ]:
#filter answers from the responses

def batch_message_filter(list_of_messages,list_of_idx) -> tuple[list[list[dict]], list[str]]:
    global answer_contributions
    extracted_answers = []
    list_of_messages_to_keep = []
    list_of_idx_to_keep = []
    for idx,messages in zip(list_of_idx,list_of_messages):
        answers = extract_boxed_texts(messages[-1]['content'])
        if answers:
            extracted_answers.extend(answers)
            for answer in answers:
                answer_contributions[answer].append(idx)
        else:
            list_of_messages_to_keep.append(messages)
            list_of_idx_to_keep.append(idx)
    return list_of_messages_to_keep, extracted_answers, list_of_idx_to_keep

In [ ]:

#execute the codes in the responses
def batch_message_execute(list_of_messages,round_idx) -> list[list[dict]]:
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'],round_idx)
        python_code = process_python_code(python_code)
        try:
            success, output = PythonREPL()(python_code)
        except Exception as e:
            output = str(e)
        messages.append({'role': 'user', 'content': output})
        print(messages[-1])
    return list_of_messages

#execute the code and generate the answer from responses
def batch_message_execute_and_get_answer(list_of_messages,round_idx) -> tuple[list[list[dict]], list[int]]:
    ans = []
    for messages in list_of_messages:
        python_code = extract_python_code(messages[-1]['content'])
        python_code = process_python_code(python_code)
        try:
            success, output = PythonREPL()(python_code)
            if success:
                patten = r'(\d+)'
                matches = re.findall(patten, output)
                if matches:
                    for match in matches:
                        ans.append(int(match)%1000)
                        ans.append(int(match)%1000) #代码权重高于自然语言，所以添加两次 
        except Exception as e:
            output = str(e)
        print(f'python code output: {output}')
    return ans

#execute code and generate answer for all elements in batch
def batch_message_list_execute_and_get_answer(list_of_messages,round_idx) -> tuple[list[list[dict]], list[int]]:
    ans = []
    for messages in list_of_messages:
        python_code_list = extract_python_code_list(messages[-1]['content'])
        for python_code in python_code_list:
            python_code = process_python_code(python_code)
            try:
                success, output = PythonREPL()(python_code)
                if success:
                    patten = r'(\d+)'
                    matches = re.findall(patten, output)
                    if matches:
                        for match in matches:
                            ans.append(int(match)%1000)
                            ans.append(int(match)%1000) 
            except Exception as e:
                output = str(e)
            print(f'python code output: {output}')
    return ans

In [ ]:
# SC-TIR control flow
def process_code(sample, restart_on_fail, last_step, check_last_n_chars=100):
    gen_text = sample["gen_texts"]
    num_python_blocks = len(re.findall(r"```python(.*?)```", gen_text, re.DOTALL))
    region_to_check = gen_text[-check_last_n_chars:]
    if num_python_blocks == 0:
        if restart_on_fail:
            print("no code has ever been generated, RESTARTING")
            sample["gen_texts"] = sample["text"]
        else:
            print("no code has ever been generated, STOP")
            sample["should_prune"] = True
            sample["has_code"] = False
        return sample
    if not gen_text.endswith("```output\n") and ("answer is" in region_to_check or "\\boxed" in region_to_check):
        num_output_blocks = len(re.findall(r"```output(.*?)```", gen_text, re.DOTALL))
        if num_output_blocks == 0:
            print("The model hallucinated the code answer")
            sample["should_prune"] = True
            return sample
        if "boxed" in region_to_check:
            try:
                answer = normalize_answer(extract_boxed_answer(region_to_check))
            except Exception:
                answer = "-1"
        else:
            answer = normalize_answer(region_to_check)
        sample["model_answers"] = answer
        return sample
    if last_step:
        return sample
    if not gen_text.endswith("```output\n"):
        print("warning: output block not found: ", gen_text[-40:])
        if restart_on_fail:
            sample["gen_texts"] = sample["text"]
        else:
            sample["should_prune"] = True
        return sample
    code_result, _ = postprocess_completion(gen_text, return_status=True, last_code_block=True)
    truncation_limit = 200
    if len(code_result) > truncation_limit:
        code_result = code_result[:truncation_limit] + " ... (output truncated)"
    sample["gen_texts"] = gen_text + f"{code_result}\n```"
    return sample


In [ ]:
import os

import pandas as pd
import polars as pl

#API for competition submission
import kaggle_evaluation.aimo_2_inference_server

In [ ]:
#correct answers for reference problems
def get_correct_answer(question):
    if 'Three airline' in question: return 79
    if 'Fred and George' in question: return 250
    if 'Triangle $ABC$' in question: return 180
    if 'Find the three' in question: return 143
    if 'We call a' in question: return 3
    if 'Let $ABC$ be' in question: return 751
    if 'For a positive' in question: return 891
    if 'For positive integers' in question: return 810
    if 'The Fibonacci numbers' in question: return 201
    if 'Alice writes all' in question: return 902
    return 0

# Inference

In [ ]:
def predict_for_question(question: str) -> int:
    global g_score
    global g_count
    global prompt_score
    global answer_contributions
    question += "\nIf the final answer is a number larger than 1000, take modulo 1000. "
    # ... (rest of the function implementation)
    sample = {
        "text": problem["text"],
        "gen_texts": problem["text"],
        "should_prune": False,
        "model_answers": "-1",
        "has_code": True,
    }
    sample = process_code(sample, restart_on_fail=True, last_step=False)
    # ... (rest of the function implementation)

In [ ]:
def main(config):
    print(f"=== Running submission with config ===\n\n{config}")
    set_seed(42)
    num_procs = os.cpu_count()
    vllm = build_vllm(config)
    sampling_params = SamplingParams(
        temperature=config.temperature,
        max_tokens=config.max_new_tokens,
        stop=["```output\n"],
        include_stop_str_in_output=True,
    )
    env, iter_test = get_kaggle_env(config)
    final_answers = []
    for test, submission in tqdm(iter_test, desc="Solving problems"):
        problem = apply_template({"prompt": test.problem.values[0]}, tokenizer=vllm.get_tokenizer(), prompt="{}")
        print(f"=== INPUT FOR PROBLEM ID {test.id.values[0]} ===\n{problem}\n")
        samples = Dataset.from_list([
            {
                "text": problem["text"],
                "gen_texts": problem["text"],
                "should_prune": False,
                "model_answers": "-1",
                "has_code": True,
            }
            for _ in range(config.num_samples)
        ])
        completed = []
        for step in range(config.num_generations):
            samples = samples.map(
                generate_batched,
                batch_size=128,
                batched=True,
                fn_kwargs={"vllm": vllm, "sampling_params": sampling_params},
                load_from_cache_file=False,
            )
            samples = samples.map(
                process_code,
                num_proc=num_procs,
                load_from_cache_file=False,
                fn_kwargs={"restart_on_fail": config.restart_on_fail, "last_step": step == (config.num_generations - 1)},
            )
            done = samples.filter(lambda x: x["should_prune"] is True, load_from_cache_file=False)
            if len(done):
                completed.append(done)
            samples = samples.filter(lambda x: x["should_prune"] is False, load_from_cache_file=False)
        completed.append(samples)
        samples = concatenate_datasets(completed)
        candidates = samples["model_answers"]
        print(f"=== CANDIDATE ANSWERS ({len(candidates)}) ===\n{candidates}\n")
        filtered = filter_answers(candidates)
        print(f"=== FILTERED ANSWERS ({len(filtered)}) ===\n{filtered}\n")
        majority = get_majority_vote(filtered)
        print(f"=== MAJORITY ANSWER (mod 1000) ===\n{majority}\n")
        submission["answer"] = majority
        env.predict(submission)
        test["model_answer"] = majority
        final_answers.append(test)
    if not config.is_submission:
        answers = env.df.merge(pd.concat(final_answers))
        answers["correct"] = answers["ground_truth"].astype(int) == answers["model_answer"].astype(int)
        print("Accuracy", answers["correct"].astype(int).mean())


In [ ]:
#predict function to solve single problem

from collections import Counter, defaultdict
g_score = 0
g_count = 0
prompt_score = Counter()
answer_contributions = defaultdict(list)
def predict_for_question(question: str) -> int:
    global g_score
    global g_count
    global prompt_score
    global answer_contributions
    question += "\nIf the final answer is a number larger than 1000, take modulo 1000. "
    if time.time() > cutoff_time: 
        return 210
    print(question)
    sample = {
        "text": problem["text"],
        "gen_texts": problem["text"],
        "should_prune": False,
        "model_answers": "-1",
        "has_code": True,
    }
    sample = process_code(sample, restart_on_fail=True, last_step=False)
    list_of_messages = [
        [
            {"role": "system", "content": thoughts[k]},
            {"role": "user", "content": question}
        ] for k in range(5)
    ]

    all_extracted_answers = []
    list_of_idx = list(range(len(list_of_messages)))
    max_round = 1
    for round_idx in range(max_round):
        print(f"round {round_idx+1}")
        list_of_messages = batch_message_generate(list_of_messages)
        #extracted_python_answer = batch_message_execute_and_get_answer(list_of_messages,round_idx)
        extracted_python_answer = batch_message_list_execute_and_get_answer(list_of_messages,round_idx)
        list_of_messages, extracted_answers, list_of_idx  = batch_message_filter(list_of_messages, list_of_idx)
        all_extracted_answers.extend(extracted_python_answer)
        all_extracted_answers.extend(extracted_answers)
        print("extracted boxed answers:",extracted_answers)
        print("extracted python answers:",extracted_python_answer)
        print("all extracted answers:",all_extracted_answers)
        if not list_of_messages:
            break
        #list_of_messages = batch_message_execute(list_of_messages,round_idx)
    answer = select_answer(all_extracted_answers)
    print("answer:",answer)
    correct_answer = get_correct_answer(question)
    print("correct answer:",correct_answer)
    g_count += 1
    if str(answer) == str(correct_answer):
        g_score += 1

    print(f"score: {g_score}/{g_count}")
    print("\n\n")
    return answer

**Predict Function provided by the hosts**

In [ ]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    id_ = id_.item(0)
    print("------")
    print(id_)
    
    question = question.item(0)
    answer = predict_for_question(question)
    print(question)
    print("------\n\n\n")
    return pl.DataFrame({'id': id_, 'answer': answer})

In [ ]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
#def predict(id_, question):
#    answer = predict_for_question(question)
#    return {'id': id_, 'answer': answer}

**Load and save the reference set to use for validation**

In [ ]:
pd.read_csv(
    '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/reference.csv'
).drop('answer', axis=1).to_csv('reference.csv', index=False)

In [ ]:
#df = pd.read_csv('/kaggle/working/reference.csv')

In [ ]:
#results = []
#for i in range(len(df)):
#    id_ = df['id'][i]
#    question = df['problem'][i]
#    result = predict(id_, question)
#    results.append(result)
#r_df = pd.DataFrame(results)
#print(r_df)

In [ ]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            'reference.csv',
        )
    )